In [86]:
import numpy as np
import pandas as pd
import math

In [87]:
##-----------------------------------------数据导入部分------------------------------------------------------------

St_s=pd.read_excel(r"C:\Users\BTE\Documents\drug\all_dataset\enzymy_ion_GPCR_nuclear\Protein sequence similarity matrix\nuclear_receptor.xlsx",head=0)
#St_s   26 rows × 26 columns

Sd_c=pd.read_excel(r"C:\Users\BTE\Documents\drug\all_dataset\enzymy_ion_GPCR_nuclear\Compound structure similarity matrix\nuclear_receptor.xlsx",head=0)
#Sd_c   54 rows × 54 columns

A=pd.read_excel(r"C:\Users\BTE\Documents\drug\all_dataset\enzymy_ion_GPCR_nuclear\Adjacency_matrix_drug-target_interaction\nuclear_receptor.xlsx",head=0)
#A      26 rows × 54 columns

#54  drugsS
#26  target

Sd_n=[]
#Sd_n   New  drug–drug similarity matrix
St_n=[]



#St_n   New  target–target similarity matrix



In [88]:
##--------------------------------------------数据处理部分------------------------------------------------------------
Sd_c=np.asmatrix(Sd_c)
St_s=np.asmatrix(St_s)
A=np.asmatrix(A) 
#Sd_n矩阵的计算(54,54)
wd=0.5
wt=0.5
Sd_n=np.zeros((len(A[0,:].T),len(A[0,:].T)))


for i in range(len(A[0,:].T)):   #i指代的是54个中的药物，按每个药物遍历
    for j in range(len(A[0,:].T)):   #j也是按照药物遍历54遍
        for k in range(len(A[:,0])):  #k是按照靶标蛋白结构遍历26遍
            if  A[k,i]==1 and  A[k,j]==1:
                Sd_n[i,j]+=1   

#St_n矩阵的计算(26,26)
St_n=np.zeros(   (len(A[:,0]),len(A[:,0]))  )
for i in range(len(A[:,0])):   #i指代的是26个中的靶标，按每个靶标遍历
    for j in range(len(A[:,0])):   #j也是按照靶标遍历26遍
        for k in range(len(A[0,:].T)):  #k是按照蛋白结构遍历54遍
            if  A[i,k]==1 and  A[j,k]==1:
                St_n[i,j]+=1
                
#将它们进行标准化处理-------------------------------------------

###-------drug
#Dd_n矩阵的计算(54,54)
Dd_n=np.diag(np.sum(Sd_n,axis=0))

#Sd_n_hat矩阵的计算(54,54)
Sd_n_hat=np.zeros((len(Sd_n),len(Sd_n)))
for i in range(len(Sd_n)):
    for j in range(len(Sd_n)):
        Sd_n_hat[i,j]=Sd_n[i,j]/(np.math.sqrt(Dd_n[i,i]*Dd_n[j,j]))
Sd_n_hat

###-------target
#Dt_n矩阵的计算(26, 26)
Dt_n=np.diag(np.sum(St_n,axis=0))
Dt_n.shape

#St_n_hat矩阵的计算(26,26)
St_n_hat=np.zeros((len(St_n),len(St_n)))
for i in range(len(St_n)):
    for j in range(len(St_n)):
        St_n_hat[i,j]=St_n[i,j]/(np.math.sqrt(Dt_n[i,i]*Dt_n[j,j]))

        
#得到最终的相似性矩阵----------------------------------------------  

Sd=wd*Sd_c+(1-wd)*Sd_n_hat
St=wt*St_s+(1-wt)*St_n_hat






In [29]:
Mdd=np.zeros((len(Sd),len(Sd)))
Mdd[:,0]=np.array(Sd[1])
Mdd

array([[0.2727275 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.53846154, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.21806302, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.25721154, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.17139802, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0125    , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [89]:
#构建异构网络----------------------------------------------------
#np.sum(St,axis=1).shape(列向量)
#np.sum(A,axis=1)==0  (列向量)

#计算Mtt
Mtt=np.zeros((len(St),len(St)))
lamda=0.5
for i in range(len(St)):
    if np.sum(A,axis=1)[i,0]==0:
        Mtt[i,:]=np.array(St[i])/np.sum(St,axis=1)[i,0]
    else:
        Mtt[i,:]=(1-lamda)*np.array(St[i])/np.sum(St,axis=1)[i,0]

#计算Mdd
Mdd=np.zeros((len(Sd),len(Sd)))
for i in range(len(Sd)):
    if np.sum(A,axis=0)[0,i]==0:           #得到行向量
        Mdd[i,:]=np.array(Sd[i])/np.sum(Sd,axis=1)[i,0]
    else:
        Mdd[i,:]=(1-lamda)*np.array(Sd[i])/np.sum(Sd,axis=1)[i,0]

#计算Mtd
Mtd=np.zeros(A.shape)
for i in range((len(St))):
    if np.sum(A,axis=1)[i,0]!=0:  
        Mtd[i,:]=np.array(A[i,:])/np.sum(A,axis=1)[i,0]
    else:
        Mtd[i,:]=np.zeros((1,len(Sd)))
        

#计算Mdt
Mdt=np.zeros(A.T.shape)
for i in range(len(Sd)):
    if np.sum(A,axis=0)[0,i]!=0:
        Mdt[i,:]=np.array(A[:,i].T)/np.sum(A,axis=0)[0,i]
    else:
        Mdt[i,:]=np.zeros((len(St),1))

#------------------得到异构网络M-----------------------------        
Mtd=np.asmatrix(Mtd)
Mtt=np.asmatrix(Mtt)
Mdd=np.asmatrix(Mdd)
Mdt=np.asmatrix(Mdt)

M=np.concatenate((np.concatenate((Mtt,Mtd) ,axis=1),np.concatenate((Mdt,Mdd) ,axis=1)),axis=0)
#M.shape=(80, 80)




In [ ]:
np.append

In [94]:
i=5
q=0.5 
v0=np.asmatrix(np.zeros((len(Sd),1)))
v0[i]=1
u0=A[:,i]/np.sum(A,0)[0,i]
p0=np.concatenate(((1-q)*u0,q*v0),axis=0)

p1[0:len(St),0].argmax()
np.linalg.norm(p1,ord=1)
p1

matrix([[0.05555556],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.05555556],
        [0.05555556],
        [0.05555556],
        [0.        ],
        [0.05555556],
        [0.05555556],
        [0.05555556],
        [0.05555556],
        [0.05555556],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.5       ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0

In [96]:
#r是restart probability
r=0.7
q=0.2
target_index=[0]
for i in range(len(Sd)):
    #假设以i作种子节点
    #即我们求药物i的关联关系

    #u0是drug neteork 的初始概率
 
    
    v0=np.asmatrix(np.zeros((len(Sd),1)))
    v0[i]=1
    u0=A[:,i]/np.sum(A,0)[0,i]
    p0=np.concatenate(((1-q)*u0,q*v0),axis=0)
    
    pt=p0
    while True:
        pt1=(1-r)*M.T*pt+r*p0
       
        if np.linalg.norm((pt-pt1),ord=1)<10**(-10):
            break
        pt=pt1    
    u_inf=pt[0:len(St),0]/(1-q)
    target_index.append(u_inf.argmax())
    
        #pt是这个醉汉第t步落在i的概率
target_index

[0,
 16,
 1,
 1,
 8,
 6,
 13,
 1,
 22,
 23,
 25,
 10,
 22,
 22,
 23,
 6,
 11,
 22,
 1,
 13,
 8,
 13,
 9,
 8,
 24,
 1,
 11,
 1,
 8,
 8,
 12,
 12,
 10,
 23,
 1,
 22,
 1,
 10,
 1,
 8,
 8,
 1,
 8,
 9,
 13,
 1,
 10,
 1,
 6,
 16,
 6,
 1,
 1,
 1,
 7]

In [275]:
class NRWRH:
    def __init__(self,Sd_c,St_s,A):
        self.Sd_c=Sd_c
        self.St_s=St_s
        
        self.A   =A
    def get_Sd_n(self,A):
        A=np.asmatrix(A) 
        """
        先把药物相似性变矩阵处理
        
        nrwrh.A[0]     ：这是个行向量，长度为54=len(A[0,:].T)
        nrwrh.A[:,0]   ：这是个列向量，长度为26=len(A[:,0])
        """
        
        Sd_n=np.zeros((len(A[0,:].T),len(A[0,:].T)))
        """
        初始化Sd_n，大小为54*54
        """
       
        for i in range(len(A[0,:].T)):   #i指代的是54个中的药物，按每个药物遍历
            for j in range(len(A[0,:].T)):   #j也是按照药物遍历54遍
                for k in range(len(A[:,0])):  #k是按照靶标蛋白结构遍历26遍
                    if  A[k,i]==1 and  A[k,j]==1:
                        Sd_n[i,j]+=1
        return Sd_n
    def get_St_n(self,A):
        A=np.asmatrix(A) 
        """
        先把靶标相似性变矩阵处理
        
        nrwrh.A[0]     ：这是个行向量，长度为54=len(A[0,:].T)
        nrwrh.A[:,0]   ：这是个列向量，长度为26=len(A[:,0])
        """
        
        St_n=np.zeros(   (len(A[:,0]),len(A[:,0]))  )
        """
        初始化St_n，大小为26*26
        """
       
        for i in range(len(A[:,0])):   #i指代的是26个中的靶标，按每个靶标遍历
            for j in range(len(A[:,0])):   #j也是按照靶标遍历26遍
                for k in range(len(A[0,:].T)):  #k是按照蛋白结构遍历54遍
                    if  A[i,k]==1 and  A[j,k]==1:
                        St_n[i,j]+=1
        return St_n
    def normalize_Sd_n(self,Sd_n):
        Sd_n=np.asmatrix(Sd_n)
        
        Dd_n=np.zeros((len(Sd_n),len(Sd_n)))
        Dd_n=np.diag(np.sum(Sd_n,axis=0))
        
        """
        Dd_n是一个对角矩阵
        
        """
        
        Sd_n_hat=np.zeros((len(Sd_n),len(Sd_n)))
        """初始化Sd_n_hat  54*54"""
        
        for Dd_n_i in range(54):
            for Dd_n_j in range(54):
                print(Dd_n[53][53])
       
        
    def normalize_St_n(self,St_n):
        St_n=np.asmatrix(St_n)
    
    #The integrated drug–drug similarity matrix  Sd
    #target–target similarity matrix St    
                                                  
    

In [8]:
nrwrh=NRWRH(Sd_c,St_s,A)
nrwrh.A=np.asmatrix(A)

Sd_n=nrwrh.get_Sd_n(nrwrh.A) 

        

nrwrh.normalize_Sd_n(Sd_n)

    


NameError: name 'NRWRH' is not defined

In [239]:
nrwrh=NRWRH(Sd_c,St_s,A)
nrwrh.A=np.asmatrix(A)

Sd_n=nrwrh.get_Sd_n(nrwrh.A) 

Dd_n=np.zeros((len(Sd_n),len(Sd_n)))
Dd_n=np.diag(np.sum(Sd_n,axis=0))
        
"""
Dd_n是一个对角矩阵
        
"""
len(Dd_n.T)

#len(Dd_n.T)
#nrwrh.normalize_Sd_n(Sd_n)

    


54